In [1]:
import h2o
h2o.init()
#h2o.cluster_info()

Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,23 secs
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,4 months and 1 day
H2O_cluster_name:,jetti
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.943 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [2]:
import numpy as np
import h2o
from h2o.estimators.kmeans import H2OKMeansEstimator

In [6]:
block = h2o.import_file('C:/Users/jetti/data/eegCompress/elimpeaksSVD001.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:

diff = np.diff(block, axis=1).astype(np.float16)
print(diff.shape)

diffFrame = h2o.H2OFrame(python_obj=diff)

In [6]:
k = 1024
max_iterations = 100

km = H2OKMeansEstimator(k=k, seed=1234, max_iterations=max_iterations)
km.train(training_frame=diffFrame)
km.scoring_history()

kmeans Model Build progress: |███████████████████████████████████████████████████| (done) 100%


,,timestamp,duration,iterations,number_of_reassigned_observations,within_cluster_sum_of_squares
0,,2025-03-03 22:07:10,0.052 sec,0.0,NaN,NaN
1,,2025-03-03 22:20:27,13 min 17.322 sec,1.0,1100366.0,963942.105208
2,,2025-03-03 22:20:49,13 min 39.044 sec,2.0,420831.0,578119.018067
3,,2025-03-03 22:21:10,14 min 0.203 sec,3.0,190331.0,553031.152717
4,,2025-03-03 22:21:30,14 min 20.703 sec,4.0,122448.0,544011.284953
...,...,...,...,...,...,...
96,,2025-03-03 22:53:26,46 min 16.965 sec,96.0,3554.0,520921.114864
97,,2025-03-03 22:53:47,46 min 38.004 sec,97.0,3526.0,520907.025286
98,,2025-03-03 22:54:08,46 min 58.484 sec,98.0,3448.0,520892.381418
99,,2025-03-03 22:54:29,47 min 19.264 sec,99.0,3350.0,520878.241169


In [7]:
nearestPoint = km.predict(diffFrame).as_data_frame(use_multi_thread=True).to_numpy().transpose()
centers = np.array(km.centers())

kmeans prediction progress: |████████████████████████████████████████████████████| (done) 100%
Export File progress: |██████████████████████████████████████████████████████████| (done) 100%


In [8]:
np.savez_compressed('/content/drive/MyDrive/NeuroResearch/Data/eegCompress/processedData/kmeansSVD001.npz', centers, nearestPoint, block[:,0])